<a href="https://colab.research.google.com/github/carolinehagood/ds4002-project3/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow
#get_ipython().system('pip install numpy==1.24.3')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 29.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.


In [1]:
import tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import requests
from io import BytesIO
import gzip
from io import BytesIO

In [3]:
import tensorflow as tf

# Check for GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
#Downloading url images
file_urls = ['https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02085936-Maltese_dog%20(3).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02088364-beagle%20(1).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02099601-golden_retriever%20(1).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02106662-German_shepherd%20(2).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02110958-pug%20(1).csv.gz']
data_frames = []
for url in file_urls:
    response = requests.get(url)
    compressed_file = BytesIO(response.content)  # Create a BytesIO object
    with gzip.open(compressed_file, 'rt') as f:  # Open it as a text file
        df = pd.read_csv(f)  # Read the CSV file into a DataFrame
        data_frames.append(df)

In [5]:
# Combine all DataFrames into a single DataFrame
data_df = pd.concat(data_frames, ignore_index=True)

# Print the DataFrame to check
print(data_df.head(1))

          0         1         2         3        4         5         6  \
0  0.141176  0.133333  0.082353  0.156863  0.14902  0.098039  0.164706   

          7         8         9  ...     49144     49145     49146     49147  \
0  0.156863  0.105882  0.172549  ...  0.223529  0.180392  0.188235  0.172549   

     49148     49149     49150     49151  breed             breed_name  
0  0.12549  0.164706  0.141176  0.098039      0  n02085936-Maltese_dog  

[1 rows x 49154 columns]


In [6]:
if data_df['breed_name'].value_counts()['n02085936-Maltese_dog'] > 100:
    data_df = data_df.drop(
        data_df[data_df['breed_name'] == 'n02085936-Maltese_dog'].iloc[100:].index
    )

In [7]:
data_df['breed_name'] = data_df['breed_name'].str.split('-').str[1]


In [8]:
print(data_df['breed'].unique())  # Check unique values
print(data_df['breed'].isna().sum())  # Check for NaN values

# Ensure breed is an integer column (convert if needed)
data_df['breed'] = data_df['breed'].astype(int)

[0 1 2 3 4]
0


In [9]:
X = data_df.iloc[:, :-2].values  # All columns except the last two
print(X.shape)

(755, 49152)


In [10]:
y = data_df['breed'].values  # Labels are from the 'breed' column
print(y.shape)

(755,)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [12]:
# Set constants

IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS = 128, 128, 3
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 4  #number of dog breeds

In [13]:
from sklearn.preprocessing import LabelEncoder

# Normalize X
X = X / 255.0  # Scale pixel values to [0, 1]
# Reshape X to (height, width, channels) for 128x128 images
X = X.reshape(-1, 128, 128, 3)  # 128x128 images with 3 color channels (RGB)


# Encode labels (y) into integers and then one-hot encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Exclude the Pug breed using its encoded integer value
exclude_label = 4  # The integer value assigned to Pug
mask = y_encoded != exclude_label  # Create a mask for all data not labeled as Pug

# Filter the data
X_filtered = X[mask]
y_filtered = y_encoded[mask]

# Update the one-hot encoded labels for filtered data
y_onehot_filtered = to_categorical(y_filtered, num_classes=NUM_CLASSES)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_onehot_filtered, test_size=0.2, random_state=42
)

# Data augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=45,  # Increased rotation
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.8, 1.2],  # Adjust brightness
    zoom_range=0.3,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [14]:
#build cnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Load the pre-trained VGG16 model, excluding the top layers
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))

# Freeze the layers of the base model so they are not trained
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = Flatten()(base_model.output)  # Flatten the output of the base model
x = Dense(256, activation='relu')(x)  # Add a dense layer
x = Dropout(0.5)(x)  # Dropout layer for regularization
x = Dense(128, activation='relu')(x)  # Add another dense layer
x = Dropout(0.5)(x)  # Dropout for regularization

# Output layer for classification
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [15]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Fit the model
history = model.fit(
    data_augmentation.flow(X_train, y_train, batch_size=64),  # Augmented training data
    validation_data=(X_test, y_test),  # Validation data
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.2831 - loss: 1.6447 - val_accuracy: 0.2411 - val_loss: 1.4798
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 93ms/step - accuracy: 0.2339 - loss: 1.7486 - val_accuracy: 0.2411 - val_loss: 1.4718
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.2546 - loss: 1.7363 - val_accuracy: 0.2411 - val_loss: 1.4642
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.2450 - loss: 1.7015 - val_accuracy: 0.2411 - val_loss: 1.4571
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.2140 - loss: 1.7159 - val_accuracy: 0.2411 - val_loss: 1.4504
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.2795 - loss: 1.6572 - val_accuracy: 0.2411 - val_loss: 1.4445


In [17]:
#unfreeze some layers of the pre-trained base model (fine-tuning)
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Step 3: Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),  # Use a lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Continue training (fine-tuning)
history_finetune = model.fit(
    data_augmentation.flow(X_train, y_train, batch_size=32),  # Augmented training data
    validation_data=(X_test, y_test),  # Validation data
    epochs=20,  # Continue for additional epochs
    initial_epoch=history.epoch[-1],  # Continue from the last trained epoch
    callbacks=[early_stopping]
)

Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.2314 - loss: 1.7019 - val_accuracy: 0.2411 - val_loss: 1.4605
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - accuracy: 0.2726 - loss: 1.4813 - val_accuracy: 0.2411 - val_loss: 1.4521
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.2535 - loss: 1.4381 - val_accuracy: 0.2411 - val_loss: 1.4382
Epoch 9/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.2635 - loss: 1.4383 - val_accuracy: 0.2411 - val_loss: 1.4292
Epoch 10/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 156ms/step - accuracy: 0.2482 - loss: 1.4193 - val_accuracy: 0.2411 - val_loss: 1.4254


In [18]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.2246 - loss: 1.4738
Test Accuracy: 24.11%
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step


In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()